In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

In [ ]:
# Load the data after preprocessing. This is how I did it the first time, but this won't work with the current preprocessing pipeline.
# train_data = np.load("train_data.npz", allow_pickle=True)
# X_train = train_data["X"]
# y_train = train_data["y"]
# file_names_train = train_data["file_names"]
#
# test_data = np.load("test_data.npz", allow_pickle=True)
# X_test = test_data["X"]
# y_test = test_data["y"]
# file_names_test = test_data["file_names"]
#
# val_data = np.load("val_data.npz", allow_pickle=True)
# X_val = val_data["X"]
# y_val = val_data["y"]
# file_names_val = val_data["file_names"]


In [ ]:
# Convert data to PyTorch tensors
# X_train_t = torch.tensor(X_train, dtype=torch.float32)
# X_val_t = torch.tensor(X_val, dtype=torch.float32)
#
# y_train_t = torch.tensor(y_train, dtype=torch.float32)
# y_val_t = torch.tensor(y_val, dtype=torch.float32)
#
## Create TensorDataset and DataLoader
# train_dataset = TensorDataset(X_train_t, y_train_t)
# val_dataset = TensorDataset(X_val_t, y_val_t)
#
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
class GRUClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super(GRUClassifier, self).__init__()

        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )

        self.fc = nn.Linear(hidden_size, 1)  # 1 logit for binary classification

    def forward(self, x):
        out, h_n = self.gru(x)

        last_hidden = h_n[-1]

        logits = self.fc(last_hidden)
        return logits.squeeze(dim=1)

In [ ]:
# Nothing below this will currently run with the new data pipeline
model = GRUClassifier(input_size=X_train.shape[2], hidden_size=64, num_layers=2)

criterion = nn.BCEWithLogitsLoss()  # for single-logit binary classification
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# I used CPU only because I don't have a GPU, but I left this here in case you do!

num_epochs = (
    5  # This took less than a minute on my CPU-only machine so you probably want more
)

for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    for batch_x, batch_y in tqdm(train_loader):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device) 

        optimizer.zero_grad()
        logits = model(batch_x)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_x.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            logits = model(batch_x)
            loss = criterion(logits, batch_y)
            val_loss += loss.item() * batch_x.size(0)

            # predicted prob -> binary predictions
            pred_prob = torch.sigmoid(logits)
            preds = (pred_prob >= 0.5).float()
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)

    val_loss = val_loss / len(val_loader.dataset)
    val_acc = correct / total

    print(
        f"Epoch [{epoch + 1}/{num_epochs}], "
        f"Train Loss: {epoch_loss:.4f}, "
        f"Val Loss: {val_loss:.4f}, "
        f"Val Acc: {val_acc:.4f}"
    )

100%|██████████| 52/52 [00:55<00:00,  1.07s/it]


Epoch [1/5], Train Loss: 0.2330, Val Loss: 0.2383, Val Acc: 0.9324


100%|██████████| 52/52 [00:41<00:00,  1.26it/s]


Epoch [2/5], Train Loss: 0.2091, Val Loss: 0.1845, Val Acc: 0.9561


100%|██████████| 52/52 [00:39<00:00,  1.32it/s]


Epoch [3/5], Train Loss: 0.1883, Val Loss: 0.1846, Val Acc: 0.9493


100%|██████████| 52/52 [00:52<00:00,  1.01s/it]


Epoch [4/5], Train Loss: 0.1800, Val Loss: 0.1740, Val Acc: 0.9561


100%|██████████| 52/52 [00:55<00:00,  1.07s/it]


Epoch [5/5], Train Loss: 0.1779, Val Loss: 0.3219, Val Acc: 0.8986


In [ ]:
test_data = np.load("test_data.npz")
X_test = test_data["X"]
y_test = test_data["y"]

# Convert to tensors
X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_t = torch.tensor(y_test, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    logits = model(X_test_t)
    test_loss = criterion(logits, y_test_t).item()

    probs = torch.sigmoid(logits)
    preds = (probs >= 0.5).float()
    accuracy = (preds == y_test_t).sum().item() / len(y_test_t)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")

Test Loss: 0.6462, Test Accuracy: 0.8007
